In [11]:
#### IMPORTS ####
using ProgressMeter
using Plots 
using Measures
using LaTeXStrings
using DelimitedFiles
# using NPZ
include("source/mOrbRG.jl")
include("source/probes.jl")
const SIZE = (750,300)
scale = 1.5
discreteCgrad = cgrad(:BuPu_3, 3, categorical = true, rev=true)
animName(orbital, num_kspace, scale) = "kondoUb_kspaceRG_$(orbital)_wave_$(num_kspace)_$(SIZE[1] * scale)x$(SIZE[2] * scale)"

animName (generic function with 1 method)

In [14]:
function getRgFlow(num_kspace_half, J_val, W_by_J_range, orbitals)
    num_kspace = 2 * num_kspace_half + 1
    k_vals = collect(range(K_MIN, K_MAX, length=num_kspace)) ./ pi
    for orbital in orbitals
        for (j, W_by_J) in collect(enumerate(W_by_J_range))
            savePath = "data/$(orbital)_$(num_kspace_half)_$(round(J_val, digits=3))_$(round(W_by_J, digits=3))"
            kondoJArray, dispersionArray = main(num_kspace_half, J_val, J_val * W_by_J, orbital)
            anim = @animate for stepIndex in 1:num_kspace_half+1
                results_norm, results_unnorm, results_bool = totalScatterProb(kondoJArray, num_kspace, stepIndex, savePath)
                # hm1 = heatmap(k_vals, k_vals, results_bool, 
                #     cmap=discreteCgrad)
                hm1 = heatmap(k_vals, k_vals, reshape(kondoJArray[3,:,stepIndex], (num_kspace, num_kspace)),
                    cmap=:matter)
                hm2 = heatmap(k_vals, k_vals, log10.(results_unnorm),
                    cmap=:matter)
                p = plot(hm1, hm2, size=SIZE, 
                    plot_title="\$W/J=$(round(W_by_J, digits=3)),\\quad \\mathrm{step=$(stepIndex)}\$", 
                    top_margin=3mm, left_margin=[5mm 10mm], bottom_margin=5mm,
                    xlabel="\$ \\mathrm{k_x/\\pi} \$", ylabel="\$ \\mathrm{k_y/\\pi} \$",
                    dpi=100 * scale
                )
                savefig(p, "fig_$(stepIndex).pdf")
            end
            gif(anim, animName(orbital, num_kspace, scale) * ".gif", fps=1)
            gif(anim, animName(orbital, num_kspace, scale) * ".mp4", fps=1)
        end
        
    end

end

getRgFlow(4, 2., [0], ["p"])

[ Info: Saved animation to /home/kirito/storage/work/TilingProject/dwaveNumerics/kondoUb_kspaceRG_p_wave_9_1125.0x450.0.gif
[ Info: Saved animation to /home/kirito/storage/work/TilingProject/dwaveNumerics/kondoUb_kspaceRG_p_wave_9_1125.0x450.0.mp4


In [36]:
M = rand(Float64, (3,3))

3×3 Matrix{Float64}:
 0.178343  0.789078  0.734908
 0.20886   0.470143  0.078514
 0.141559  0.262283  0.324695

In [37]:
reshape(M, (9))

9-element Vector{Float64}:
 0.17834259861778656
 0.20886034934090458
 0.14155922935059784
 0.7890775997954708
 0.4701427375830358
 0.26228313119646673
 0.7349079484054621
 0.07851398799836973
 0.3246945765939532